<a href="https://colab.research.google.com/github/nw93929/DS2002FinalProject/blob/main/SCRIPTS/animatedmap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/nw93929/DS2002FinalProject

Cloning into 'DS2002FinalProject'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 81 (delta 27), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 16.26 MiB | 13.78 MiB/s, done.
Resolving deltas: 100% (27/27), done.


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import geopandas as gpd
from folium.plugins import TimeSliderChoropleth
import ipywidgets as widgets
from ipywidgets import interact
from folium import LinearColormap

# Load the dataset
geo_data = gpd.read_file('/content/DS2002FinalProject/Data/counties.geojson.txt')
winners_data = pd.read_excel('/content/DS2002FinalProject/Data/WinningPartyByCounties.xlsx')
geo_data['NAME'] = geo_data['NAME'].str.upper()
merged_data = geo_data.merge(winners_data, left_on='NAME', right_on='county_name')
states_to_keep = ["AZ", "MI", "GA", "WI", "NV", "NC", "PA"] #only including the swing states in 2020
winners_data = winners_data[winners_data['state_po'].isin(states_to_keep)]
winners_data.head()

# format party codes 0 for DEM 1 for REP
merged_data['party_code'] = merged_data['party'].astype('category').cat.codes
merged_data2 = merged_data.groupby('state')['party_code'].mean().reset_index()
merged_data = merged_data.merge(merged_data2, left_on='state', right_on='state')

In [3]:
# separate merged data into 2012-2020 results
merged_data2020 = merged_data[merged_data['year'] == 2020]
merged_data2016 = merged_data[merged_data['year'] == 2016]
merged_data2012 = merged_data[merged_data['year'] == 2012]

In [6]:
merged_data2012.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,...,county_fips,office,candidate,party,candidatevotes,totalvotes,version,mode,party_code_x,party_code_y
0,37,181,01008591,0500000US37181,37181,VANCE,06,653705784,42187365,"POLYGON ((-78.49778 36.51448, -78.45728 36.541...",...,37181,US PRESIDENT,BARACK OBAMA,DEMOCRAT,13323,20854,20220315,TOTAL,0,0.773116
3,50,011,01461762,0500000US50011,50011,FRANKLIN,06,1641633748,150930318,"POLYGON ((-73.25632 44.75944, -73.19279 44.908...",...,13119,US PRESIDENT,MITT ROMNEY,REPUBLICAN,6114,7762,20220315,TOTAL,1,0.777033
4,50,011,01461762,0500000US50011,50011,FRANKLIN,06,1641633748,150930318,"POLYGON ((-73.25632 44.75944, -73.19279 44.908...",...,37069,US PRESIDENT,MITT ROMNEY,REPUBLICAN,14603,28389,20220315,TOTAL,1,0.773116
5,50,011,01461762,0500000US50011,50011,FRANKLIN,06,1641633748,150930318,"POLYGON ((-73.25632 44.75944, -73.19279 44.908...",...,42055,US PRESIDENT,MITT ROMNEY,REPUBLICAN,43260,63078,20220315,TOTAL,1,0.859240
12,16,065,00394803,0500000US16065,16065,MADISON,06,1215396379,10500827,"POLYGON ((-111.97685 43.92738, -111.83759 43.8...",...,13195,US PRESIDENT,MITT ROMNEY,REPUBLICAN,8443,11104,20220315,TOTAL,1,0.777033


In [10]:
# Create Folium map 2012
map_results2012 = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# color scale (higher means republican)
color_scale = LinearColormap(
    colors=['red', 'white', 'blue'],  # reverse the colors from blue-white-red
    vmin=merged_data2020['party_code_x'].min(),  # set minimum value for the scale
    vmax=merged_data2020['party_code_x'].max()   # set maximum value for the scale
)

# Add choropleth layer 2012
folium.Choropleth(
    geo_data=geo_data,
    data=merged_data2012,
    columns=['NAME', 'party_code_x'],
    fill_color=color_scale,
    fill_opacity=0.7,
    line_opacity=0.2,
    key_on='feature.properties.NAME').add_to(map_results2012)

# Create Folium map 2016
map_results2016 = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# Add choropleth layer 2016
folium.Choropleth(
    geo_data=geo_data,
    data=merged_data2016,
    columns=['NAME', 'party_code_x'],
    fill_color=color_scale,
    fill_opacity=0.7,
    line_opacity=0.2,
    key_on='feature.properties.NAME').add_to(map_results2016)

# Create Folium map 2020
map_results2020 = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

# Add choropleth layer 2020
folium.Choropleth(
    geo_data=geo_data,
    data=merged_data2020,
    columns=['NAME', 'party_code_x'],
    fill_color=color_scale,
    fill_opacity=0.7,
    line_opacity=0.2,
    key_on='feature.properties.NAME').add_to(map_results2020)

# Map creation function
def create_map(year):
    if year == 2012:
        m = map_results2012
    elif year == 2016:
        m = map_results2016
    elif year == 2020:
        m = map_results2020

    return m

# Create the widget for selecting year
year_dropdown = widgets.Dropdown(
    options=[2012, 2016, 2020],
    value=2020,
    description='Year:',
    disabled=False
)

# Create a function to update the map based on selected year
def update_map(year):
    # Clear the output widget and display the new map
    display(create_map(year))

# Use interact to connect the dropdown to the function
interact(update_map, year=year_dropdown)

ValueError: color should be a string, not a <class 'branca.colormap.LinearColormap'>.